In [3]:
import re
from os import system
import socket

def get_file_table(path):
    system("aws s3 ls %s > awsfiles.txt"%path)
    pat=re.compile(r'(.*)\.([^\.]*)$')

    T={}
    with open('awsfiles.txt','r') as files:
        for file in files.readlines():
            parts=file.strip().split()
            if len(parts)!=4:
                continue
            filename=parts[3]
            if not 'lossless' in filename:
                continue
            m=pat.match(filename)
            if m:
                file,ext= m.groups()
                info=(ext,parts[0]+' '+parts[1])
                if file in T:
                    T[file].append(info)
                else:
                    T[file]=[info]
            else:
                print(filname,'no match')
    return T


def find_and_lock(stack_directory):
    T=get_file_table(stack_directory)

    answer=''
    
    while len(answer)==0:
        #find a file without a lock
        found=False
        for item in T.items():
            if len(item[1])==1:
                found=True
                break
        if not found:
            return None
        
        filename=item[0]
        extensions=item[1]

        #create a lock
        hostname=socket.gethostname().replace('.','+')
        flagname=filename+'.lock-'+hostname
        open(flagname,'w').write(flagname+'\n')

        command='aws s3 cp %s %s%s'%(flagname,stack_directory,flagname)
        system(command)

        # check to make sure that there is only one lock.
        T=get_file_table(stack_directory)
        extensions=T[filename]
        if len(extensions)==2:
            return filename
        
        # translation of date for better handling of two machines putting locks 
        # at nearly the same time
        # comparing the time stamps of the locks can be used to resolve who was firstand should continue
        # and who should look for another file.
        # from datetime import datetime
        # d1=datetime.strptime('2018-08-28 21:16:34','%Y-%m-%d %H:%M:%S')

In [1]:
from os import mkdir

stack_directory='s3://mousebraindata-open/MD657/'
local_data='../../data'

In [ ]:
stem=find_and_lock(stack_directory)

def run(command):
    print('cmd=',command)
    system(command)

In [67]:
run('aws s3 cp %s%s.jp2 %s/%s.jp2'%(stack_directory,stem,local_data,stem))
run('kdu_expand -i %s/%s.jp2 -o %s/%s.tif'%(local_data,stem,local_data,stem))
mkdir(local_data+'/tiles')
run('convert %s/%s -crop 1000x1000  +repage  +adjoin  %s/tiles/tiles_%02d.tif'\
    %(local_data,stem,local_data))

In [100]:
!ls -l ../../data/

total 6505520
-rw-r--r--     1 yoavfreund  staff  1132724224 Sep  8 16:58 MD657-F10-2017.02.17-19.35.42_MD657_3_0030_lossless.jp2.49d94Bcb
-rw-r--r--@    1 yoavfreund  staff  2159505277 Jul  1 17:18 MD657-F23-2017.02.17-23.13.17_MD657_2_0068_lossless.jp2
-rw-r--r--@    1 yoavfreund  staff    22260806 Sep  6 15:31 MD657-N71-2017.02.22-18.34.41_MD657_1_0211_lossy.jp2
-rw-r--r--     1 yoavfreund  staff     6885462 Aug 30 13:42 md657.tiff
drwxr-xr-x  2705 yoavfreund  staff       86560 Sep  8 12:45 tifTiles
drwxr-xr-x  1154 yoavfreund  staff       36928 Aug 29 19:28 tiles
-rw-r--r--     1 yoavfreund  staff     3060994 Sep  7 18:27 tiles_612_extracted.pkl


In [83]:
formatted='2018-08-28 21:16:34'
from datetime import datetime
d1=datetime.strptime('2018-08-28 21:16:34','%Y-%m-%d %H:%M:%S')
d2=datetime.strptime('2018-08-28 21:16:38','%Y-%m-%d %H:%M:%S')
d1<d2

True

In [4]:
import socket
from os import getpid,system,mkdir
from os.path import isfile
from sys import argv
import re
import socket
import argparse

In [2]:
!ls

Check extracted.ipynb
Finding cell in window.ipynb
ManageEC2Jobs.ipynb
awsfiles.txt
files.txt
flag
generate Normalized Patches.ipynb
initial exploration, generated window.pkl.ipynb
log
psutil.ipynb
read image and break into tiles-Copy1..ipynb
read image and break into tiles..ipynb


In [20]:
%%writefile watchdog.py
#!/usr/bin/env python3

from os.path import isfile,getmtime
from glob import glob
from time import sleep,time

stack='s3://mousebraindata-open/MD657'
local_data='/dev/shm/data'
exec_dir='/home/ubuntu/shapeology_code/scripts'

def run(command):
    print('cmd=',command)
    system(command)

def Last_Modified(file_name):
    try:
        mtime = getmtime(file_name)
    except OSError:
        mtime = 0
    return(mtime)

if __name__=='__main__':
    Recent=False
    for logfile in glob(exec_dir+'/Controller*.log'):
        if time() - Last_Modified(logfile) <600: # allow 10 minute idle
            Recent=True
            break
    if(not Recent):
        command='{0}/Controller.py {1} {2} &> {0}/Controller-{3}.log &'\
        .format(exec_dir,stack,local_data,int(time()))
    run(command)


Writing watchdog.py


In [18]:
def run(command):
    print('cmd=',command)
    out=system(command)
    return out

In [10]:
#clear locks
stack_directory='s3://mousebraindata-open/MD657/'
T=get_file_table(stack_directory)

In [11]:
for item in T.items():
    for ext in item[1]:
        if 'lock' in ext[0]:
            filename=stack_directory+item[0]+'.'+ext[0]
            run('aws s3 rm %s'%filename)

cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F15-2017.02.17-20.46.06_MD657_3_0045_lossless.lock-ip-172-30-1-84
cmd= aws s3 rm s3://mousebraindata-open/MD657/MD657-F88-2017.02.23-17.31.36_MD657_2_0263_lossless.lock-ip-172-30-1-84


In [7]:
T

{'MD657-F1-2017.02.17-17.39.26_MD657_1_0001_lossless': [('jp2',
   '2018-08-28 21:16:34')],
 'MD657-F1-2017.02.17-17.39.26_MD657_2_0002_lossless': [('jp2',
   '2018-08-28 21:16:33')],
 'MD657-F1-2017.02.17-17.39.26_MD657_3_0003_lossless': [('jp2',
   '2018-08-28 21:16:33')],
 'MD657-F10-2017.02.17-19.35.42_MD657_1_0028_lossless': [('jp2',
   '2018-08-28 21:16:34')],
 'MD657-F10-2017.02.17-19.35.42_MD657_3_0030_lossless': [('jp2',
   '2018-08-28 21:16:36')],
 'MD657-F10-2017.02.21-17.57.42_MD657_1_0028_lossless': [('jp2',
   '2018-08-28 21:16:51')],
 'MD657-F10-2017.02.21-17.57.42_MD657_2_0029_lossless': [('jp2',
   '2018-08-28 21:16:52')],
 'MD657-F10-2017.02.21-17.57.42_MD657_2_0029_lossless_patches': [('tgz',
   '2018-09-10 15:40:04')],
 'MD657-F10-2017.02.21-17.57.42_MD657_3_0030_lossless': [('jp2',
   '2018-08-28 21:16:54')],
 'MD657-F11-2017.02.17-19.39.47_MD657_1_0031_lossless': [('jp2',
   '2018-08-28 21:17:01')],
 'MD657-F11-2017.02.17-19.39.47_MD657_2_0032_lossless': [('jp2',
